### kcbert로 감정분석
beomi/kcbert-base

In [22]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch

# Hugging Face에서 kcBERT 모델과 토크나이저를 불러옵니다.
model_name = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 감정 분류를 위한 3개 라벨 (긍정, 부정, 중립)

# CSV 파일 읽기
df = pd.read_csv("./R&B-Soul.csv")

# b 열에 있는 문장 리스트
sentences = df['lyrics'].tolist()

# 문장들을 토크나이징
tokens = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# 모델을 평가 모드로 설정
model.eval()

# 감정 예측
with torch.no_grad():
    outputs = model(**tokens)
    kcbert_pred = torch.argmax(outputs.logits, dim=-1)


C:\Users\yuvnn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### kcelectrica로 감정분석
beomi/KcELECTRA-base-v2022

In [23]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# 감정 레이블 (행복, 슬픔, 분노, 당황, 중립)
emotion_labels = ['행복', '슬픔', '분노', '당황', '중립']

# Hugging Face에서 KcELECTRA 기반 감정 분석 모델과 토크나이저 불러오기
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# CSV 파일 불러오기
df = pd.read_csv("./R&B-Soul.csv")

# b 열에 있는 문장 리스트
sentences = df['lyrics'].tolist()

# 문장들을 토크나이징
tokens = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# 모델을 평가 모드로 설정
model.eval()

# 감정 예측
with torch.no_grad():
    outputs = model(**tokens)
    logits = outputs.logits
    kcelectrica_probs = F.softmax(logits, dim=-1)  # 소프트맥스를 사용하여 확률 계산
    kcelectrica_pred = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 감정을 예측

# 예측된 숫자 라벨을 감정 레이블로 변환
emo_kcelectrica = [emotion_labels[prediction] for prediction in kcelectrica_pred]

# 각 감정에 대한 확률을 퍼센티지로 변환
kcelectrica_pct = kcelectrica_probs * 100  # 확률을 퍼센티지로 변환




C:\Users\yuvnn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### csv에 저장

In [24]:
# 예측 결과를 새로운 열에 추가
# 예측 결과를 새로운 열에 추가
df['kcbert001'] = kcbert_pred.numpy()
df['kcelectrica001'] = emo_kcelectrica
df['kcelectrica001-confidence'] = [f'{kcelectrica_pct[i][prediction]:.2f}%' for i, prediction in enumerate(kcelectrica_pred)]  # 예측 감정의 확률

# 감정 분석 결과가 추가된 CSV 파일을 저장 (덮어쓰기)
df.to_csv("./R&B-Soul_output.csv", index=False,encoding='utf-8-sig')

print("감정 분석 및 확률 결과 저장 완료.")

감정 분석 및 확률 결과 저장 완료.
